# 03 - Recommender System

In [1]:
# imports
import pandas as pd
import numpy as np
import time
import sqlite3
import pickle

from scipy import sparse # cut down on memory size
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine

pd.options.display.max_columns = 35

___

In [2]:
# read in the data
df = pd.read_csv('../Data/cards_cleaned.csv')
df.head()

,name,layout,colors,color_identity,mana_cost,cmc,type_line,card_type,super_type,sub_type,oracle_text,oracle_text_token,legalities,rarity,power,toughness,loyalty,card_faces,activated_ability,triggered_ability,oracle_text_back,oracle_text_back_token,colors_back,power_back,toughness_back,loyalty_back,card_type_back,super_type_back,sub_type_back,mana_cost_back
0,Static Orb,normal,[],[],{3},3.0,Artifact,Artifact,NONE,NONE,"As long as Static Orb is untapped, players can...",as long as static orb is untapped players can'...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,Sensory Deprivation,normal,['U'],['U'],{U},1.0,Enchantment — Aura,Enchantment,NONE,Aura,Enchant creature Enchanted creature gets -3/-0.,enchant creature enchanted creature gets -3/-0,pioneer modern legacy pauper vintage penny com...,common,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,Road of Return,normal,['G'],['G'],{G}{G},2.0,Sorcery,Sorcery,NONE,NONE,Choose one — • Return target permanent card fr...,choose one return target permanent card from y...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,Storm Crow,normal,['U'],['U'],{1}{U},2.0,Creature — Bird,Creature,NONE,Bird,Flying (This creature can't be blocked except ...,flying this creature can't be blocked except b...,modern legacy pauper vintage penny commander duel,common,1,2,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,Walking Sponge,normal,['U'],['U'],{1}{U},2.0,Creature — Sponge,Creature,NONE,Sponge,{T}: Target creature loses your choice of flyi...,{t} target creature loses your choice of flyin...,legacy vintage commander duel,uncommon,1,1,NONE,NONE,1.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [3]:
df.shape

(19264, 30)

___

## Oracle text recommender system
To start I want to build a recommender system that just looks at oracle text. In order to do that I need to combine both oracle_text and oracle_text_back into one single column

In [4]:
df['oracle_combined'] = df['oracle_text_token'] + " " + df['oracle_text_back_token']

In [5]:
# start by isolating the name of the card and it's oracle_text
oracle = df['oracle_combined']

# vectorize all our words
cvec = CountVectorizer(stop_words=['none'],
                      min_df=2,
                      max_df=.99,
                      ngram_range=(1,6),
                      token_pattern="[a-zA-Z{}+'0-9-/−]+") # we should use the same RegEx to keep certain characters together 

oracle_vec = cvec.fit_transform(oracle)

# convert to a dataframe so we can use this later on as well
converted_df = pd.DataFrame(oracle_vec.toarray(), columns=cvec.get_feature_names(), index=df['name'])

In [6]:
converted_df.shape

(19264, 136319)

This is a whole lot of features to examine so I'm going to use Variance Threshold to drop some n_gram columns that only exist in a small number of cards

In [7]:
selector = VarianceThreshold(.001)
new_array = selector.fit_transform(converted_df)

new_array.shape

(19264, 14858)

In [8]:
# get the list of columns kept
vt_list = converted_df.columns[selector.get_support()]

# update our converted_df with just the important columns
converted_df = converted_df[vt_list]

For the recommender system to work efficiently we need to convert the data back into a sparse matrix with the new names as the index

In [9]:
sparse_df = sparse.csr_matrix(converted_df)

In [10]:
# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

In [11]:
rec.shape

(19264, 19264)

In [12]:
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,Palinchron,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,...,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Freyalise's Winds,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,1.000000,1.000000,0.976967,1.000000,0.916955,0.976224,1.000000,1.000000,1.00000,1.000000,0.897467,0.979617,0.933296,0.982608,0.962356,1.000000,...,0.977805,0.987648,0.800691,0.983457,0.975825,0.980091,1.000000,1.000000,0.922746,0.913040,1.000000,1.000000,1.0,0.985500,1.000000,0.944326,1.000000
Sensory Deprivation,1.000000,0.000000,1.000000,0.939834,0.922326,1.000000,1.000000,0.840510,0.952886,1.00000,1.000000,0.625030,0.946757,1.000000,0.954569,1.000000,1.000000,...,1.000000,1.000000,0.869842,0.956786,1.000000,0.895990,1.000000,0.943989,1.000000,1.000000,1.000000,1.000000,1.0,0.962122,0.947074,0.927285,1.000000
Road of Return,1.000000,1.000000,0.000000,1.000000,0.926295,1.000000,0.917493,0.984866,0.910586,1.00000,0.952637,0.979668,0.979791,0.884262,0.793073,0.900469,0.869811,...,0.966991,0.859158,0.909427,0.942592,0.856182,0.970391,0.800997,0.925591,0.974468,0.948268,0.911534,0.694050,1.0,0.928115,0.949779,0.855100,1.000000
Storm Crow,0.976967,0.939834,1.000000,0.000000,0.920554,1.000000,1.000000,0.979609,0.927716,1.00000,1.000000,0.958908,0.959156,0.955445,0.976766,0.907804,1.000000,...,0.940700,0.991749,0.789214,0.966850,0.983852,0.946808,1.000000,1.000000,0.982799,0.988383,0.981662,1.000000,1.0,0.970944,0.959400,0.972110,0.962602
Walking Sponge,1.000000,0.922326,0.926295,0.920554,0.000000,1.000000,0.938493,0.552477,0.953341,0.90755,1.000000,0.946950,0.929695,0.971240,0.850026,0.794411,0.971693,...,0.732054,0.914787,0.942711,0.957204,0.916612,0.931330,0.961538,0.889060,1.000000,1.000000,0.917141,0.960579,1.0,0.812441,0.912643,0.771960,1.000000


In [13]:
# now to test the recommender system
rec_df['Shock'].sort_values()[0:11]

name
Shock                0.000000
Tarfire              0.000000
Unyaro Bee Sting     0.000000
Bee Sting            0.000000
Magma Jet            0.058487
Deadapult            0.133975
Moonglove Extract    0.133975
Shock Troops         0.133975
Goblin Test Pilot    0.149037
Blood Rites          0.149037
Seismic Assault      0.149037
Name: Shock, dtype: float64

In [14]:
rec_df['Static Orb'].sort_values()[0:11]

name
Static Orb                0.000000
Winter Orb                0.099500
Imi Statue                0.360979
Damping Field             0.360979
Smoke                     0.406144
Stoic Angel               0.442914
Storage Matrix            0.540666
Castle Raptors            0.543250
Kill Switch               0.578152
Giant Tortoise            0.594660
Juniper Order Advocate    0.620690
Name: Static Orb, dtype: float64

In [15]:
rec_df['Prized Amalgam'].sort_values()[0:11]

name
Prized Amalgam            0.000000
Bone Dragon               0.367289
Reassembling Skeleton     0.374905
Footsteps of the Goryo    0.378368
Chronosavant              0.408758
Scrapheap Scrounger       0.409055
Apprentice Necromancer    0.414345
Wake the Dead             0.418516
Despoiler of Souls        0.418516
Ghoulsteed                0.424205
Cauldron Dance            0.425656
Name: Prized Amalgam, dtype: float64

In [16]:
rec_df['Wrath of God'].sort_values()[0:11]

name
Damnation                  0.000000
Wrath of God               0.000000
Perish                     0.117647
Winds of Rath              0.172660
Shatterstorm               0.185908
Plague Wind                0.207882
Abu Ja'far                 0.238958
Jokulhaups                 0.240743
Retribution of the Meek    0.344064
Do or Die                  0.348305
Obliterate                 0.356079
Name: Wrath of God, dtype: float64

In [17]:
rec_df['Grizzly Bears'].sort_values()[0:11] # vanilla creature

name
Grizzly Bears              0.0
Static Orb                 1.0
Scaled Behemoth            1.0
Dwarven Armory             1.0
Duskdale Wurm              1.0
Myojin of Infinite Rage    1.0
Anthem of Rakdos           1.0
Ash Zealot                 1.0
Devils' Playground         1.0
Capashen Templar           1.0
Primordial Wurm            1.0
Name: Grizzly Bears, dtype: float64

This is a great start! Now I want to add the numerical features and see the results
___

## Adding numerical data to our features

In [18]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
card_faces                 object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back

In [19]:
# we already have a df for the oracle text so we can just use that one and add to it.
converted_df['cmc'] = df['cmc'].values
converted_df['activated_ability'] = df['activated_ability'].values
converted_df['triggered_ability'] = df['triggered_ability'].values

# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,Palinchron,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,...,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Freyalise's Winds,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.885292,0.911879,0.863296,0.853265,0.539805,0.720115,0.849812,0.953171,0.595071,0.844143,0.763221,0.831784,0.587970,0.854321,0.934378,0.774718,...,0.867995,0.864715,0.706802,0.824260,0.758095,0.880962,0.886683,0.757875,0.804900,0.759053,0.831684,0.746309,0.513336,0.912650,0.734547,0.814129,0.748688
Sensory Deprivation,0.885292,0.000000,0.957321,0.886499,0.857866,0.794262,0.872918,0.781782,0.931959,0.803884,0.924515,0.582248,0.877794,0.817797,0.894167,0.984109,0.890891,...,0.945200,0.939519,0.817426,0.882149,0.891852,0.851750,0.945118,0.835827,0.937006,0.917626,0.918481,0.877132,0.764298,0.927476,0.828580,0.864968,0.878284
Road of Return,0.911879,0.957321,0.000000,0.956403,0.877161,0.841948,0.834034,0.930150,0.895458,0.849339,0.898519,0.924488,0.924895,0.790042,0.756089,0.890129,0.804420,...,0.926328,0.819956,0.867535,0.886830,0.792297,0.933565,0.768111,0.846853,0.927410,0.889258,0.855963,0.650757,0.818929,0.902500,0.860081,0.811981,0.906495
Storm Crow,0.863296,0.886499,0.956403,0.000000,0.854809,0.789837,0.870185,0.907120,0.907327,0.799666,0.922891,0.887040,0.887649,0.787290,0.913512,0.894487,0.888544,...,0.888043,0.930495,0.740971,0.889646,0.875717,0.899041,0.943937,0.880212,0.919564,0.905337,0.900073,0.874489,0.759228,0.935177,0.835837,0.905169,0.844583
Walking Sponge,0.853265,0.857866,0.877161,0.854809,0.000000,0.736819,0.788667,0.488234,0.912961,0.665503,0.903439,0.858544,0.828041,0.733629,0.769850,0.786557,0.837165,...,0.684550,0.845264,0.870250,0.849244,0.792486,0.873572,0.894692,0.759988,0.919418,0.894627,0.822724,0.811392,0.698489,0.768068,0.767014,0.697717,0.844300


In [20]:
# now to test the recommender system
rec_df['Shock'].sort_values()[0:11]

name
Shock                0.000000e+00
Tarfire              4.440892e-16
Magma Jet            6.580127e-02
Bee Sting            1.237505e-01
Unyaro Bee Sting     1.237505e-01
Moonglove Extract    1.699426e-01
Deadapult            1.699426e-01
Seal of Fire         1.708438e-01
Ember Hauler         1.711375e-01
Orcish Vandal        1.711375e-01
Arc Trail            1.776544e-01
Name: Shock, dtype: float64

In [21]:
rec_df['Static Orb'].sort_values()[0:11]

name
Static Orb          0.000000
Winter Orb          0.095126
Damping Field       0.264233
Imi Statue          0.264233
Stoic Angel         0.315962
Smoke               0.331901
Castle Raptors      0.371254
Wardscale Dragon    0.423140
Skyrider Trainee    0.437517
Storage Matrix      0.449177
Mungha Wurm         0.454499
Name: Static Orb, dtype: float64

In [22]:
rec_df['Prized Amalgam'].sort_values()[0:11]

name
Prized Amalgam            0.000000
Footsteps of the Goryo    0.352702
Reassembling Skeleton     0.362452
Bone Dragon               0.368833
Scrapheap Scrounger       0.392484
Apprentice Necromancer    0.397741
Cauldron Dance            0.398335
Wake the Dead             0.398827
Despoiler of Souls        0.401777
Skyfire Phoenix           0.420695
Stitchwing Skaab          0.424360
Name: Prized Amalgam, dtype: float64

In [23]:
rec_df['Wrath of God'].sort_values()[0:11]

name
Wrath of God       0.000000
Damnation          0.000000
Perish             0.078235
Winds of Rath      0.085894
Shatterstorm       0.093307
Jokulhaups         0.116040
Plague Wind        0.116612
Obliterate         0.167576
Catastrophe        0.182008
Day of Judgment    0.183503
Child of Alara     0.187596
Name: Wrath of God, dtype: float64

In [24]:
rec_df['Jace, the Mind Sculptor'].sort_values()[0:11]

name
Jace, the Mind Sculptor    0.000000
Coral Fighters             0.374088
Voyage's End               0.389159
Brainstorm                 0.413580
Riverwise Augur            0.415442
Select for Inspection      0.432234
Eye Spy                    0.434878
Anchor to the Aether       0.438873
Precognition               0.443417
Dream Cache                0.450290
Cavalier of Gales          0.453835
Name: Jace, the Mind Sculptor, dtype: float64

In [25]:
rec_df['Grizzly Bears'].sort_values()[0:11] # vanilla creature

name
Jedit Ojanen             0.0
Barktooth Warbeard       0.0
Metallic Sliver          0.0
Fire Elemental           0.0
Harrier Naga             0.0
Cabal Evangel            0.0
Old Ghastbark            0.0
Knight of New Benalia    0.0
Woolly Thoctar           0.0
Wu Infantry              0.0
Armored Whirl Turtle     0.0
Name: Grizzly Bears, dtype: float64

___

## Non-numerical data
Now I have to convert the non-numerical data into numerical data

### Power / Toughness / Loyalty

Through some outside reasearch on ScryFall and how other sources interpret * s in power and toughness, I'm going to impute the * s as 0 and * +1s as 1 for both power and toughness. Same is true for loyalty

In [26]:
# power
zero_power_index = df.loc[(df['power'] == '*')].index
zero_power_back_index = df.loc[(df['power_back'] == '*')].index
df.loc[zero_power_index, 'power'] = 0
df.loc[zero_power_back_index, 'power_back'] = 0

one_power_index = df.loc[(df['power'] == '1+*')].index
df.loc[one_power_index, 'power'] = 1

two_power_index = df.loc[(df['power'] == '2+*')].index
df.loc[two_power_index, 'power'] = 2

# toughness
zero_toughness_index = df.loc[(df['toughness'] == '*')].index
zero_toughness_back_index = df.loc[(df['toughness_back'] == '*')].index
df.loc[zero_toughness_index, 'toughness'] = 0
df.loc[zero_toughness_back_index, 'toughness_back'] = 0

one_toughness_index = df.loc[(df['toughness'] == '1+*')].index
df.loc[one_toughness_index, 'toughness'] = 1

two_toughness_index = df.loc[(df['toughness'] == '2+*')].index
df.loc[two_toughness_index, 'toughness'] = 2

seven_toughness_index = df.loc[(df['toughness'] == '7-*')].index
df.loc[seven_toughness_index, 'toughness'] = 7

# loyalty
zero_loyalty_index = df.loc[(df['loyalty'] == 'X')].index
zero_loyalty_back_index = df.loc[(df['loyalty_back'] == 'X')].index
df.loc[zero_loyalty_index, 'loyalty'] = 0
df.loc[zero_loyalty_back_index, 'loyalty_back'] = 0

In [27]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
card_faces                 object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back

In [28]:
# add the newly made numerical columns to the converted df
converted_df['power'] = df['power'].values
converted_df['power_back'] = df['power_back'].values
converted_df['toughness'] = df['toughness'].values
converted_df['toughness_back'] = df['toughness_back'].values
converted_df['loyalty'] = df['loyalty'].values
converted_df['loyalty_back'] = df['loyalty_back'].values

In [29]:
converted_df.head()

,+0/+1,+0/+1 until,+0/+1 until end,+0/+1 until end of,+0/+1 until end of turn,+0/+2,+0/+2 until,+0/+2 until end,+0/+2 until end of,+0/+2 until end of turn,+0/+3,+1,+1/+0,+1/+0 and,+1/+0 and gains,+1/+0 and has,+1/+0 counters,...,{w} {t} tap target,{w}{u}{b}{r}{g},{w}{w},{x},{x} {t},−1,−2,−3,−6,−7,−8,cmc,activated_ability,triggered_ability,power_back,toughness_back,loyalty_back
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,3.0,0.0,0.0,NONE,NONE,NONE
Sensory Deprivation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,NONE,NONE,NONE
Road of Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NONE,NONE,NONE
Storm Crow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NONE,NONE,NONE
Walking Sponge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0,0.0,NONE,NONE,NONE


___

In [30]:
# set the index of df to be the same as the converted_df; the name of the card
df = df.set_index(df['name']).drop(columns='name')

In [31]:
# essentially makeing dummy variables for non-numerical data

# initialize some lists to check
wburg = ['B', 'G', 'R', 'U', 'W']

card_types = ['Creature', 'Instant', 'Enchantment', 'Sorcery', 'Artifact', 'Land', 'Planeswalker', 'Tribal']

subtypes = list(set(" ".join(df['sub_type'].value_counts().index).split()))
subtypes.remove('NONE')

super_types = list(set(" ".join(df['super_type'].value_counts().index).split()))
super_types.remove('NONE')

formats = list(set(" ".join(df['legalities'].value_counts().index).split()))
formats.remove('NONE')

rarities = ['common', 'uncommon', 'rare', 'mythic']


# timer and progress checker
t0 = time.time()
counter = 0

for counter, index in enumerate(converted_df.index):
    # colors
    for color in wburg:
        converted_df.loc[index, 'colors_' + color] = (color in df.loc[index, 'colors']) * 1
        converted_df.loc[index, 'color_identity_' + color] = (color in df.loc[index, 'color_identity']) * 1
    
    # card_types
    for ctype in card_types:
        converted_df.loc[index, 'card_type_' + ctype] = (ctype in df.loc[index, 'card_type']) * 1
        
    # sub_types
    for stype in subtypes:
        converted_df.loc[index, 'sub_type_' + stype] = (stype in df.loc[index, 'sub_type']) * 1
        
    # super_type
    for st in super_types:
        converted_df.loc[index, 'super_type_' + st] = (st in df.loc[index, 'super_type']) * 1
    
    # legalities
    for form in formats:
        converted_df.loc[index, 'legalities_' + form] = (form in df.loc[index, 'legalities']) * 1
    
    # rarity
    for r in rarities:
        converted_df.loc[index, 'rarity_' + r] = (r in df.loc[index, 'rarity']) * 1
    
    # progress checker
    if counter % 1000 == 0:
        print(f'Converted {counter} cards out of a total of {len(converted_df.index)}')
        print(f'mins: {(time.time() - t0)/60}')
        print('-------------------------------')

Converted 0 cards out of a total of 19264
mins: 0.2933765689531962
-------------------------------
Converted 1000 cards out of a total of 19264
mins: 3.593815815448761
-------------------------------
Converted 2000 cards out of a total of 19264
mins: 6.885796197255453
-------------------------------
Converted 3000 cards out of a total of 19264
mins: 10.315735614299774
-------------------------------
Converted 4000 cards out of a total of 19264
mins: 13.686725266774495
-------------------------------
Converted 5000 cards out of a total of 19264
mins: 16.97920649846395
-------------------------------
Converted 6000 cards out of a total of 19264
mins: 20.273757231235503
-------------------------------
Converted 7000 cards out of a total of 19264
mins: 23.56416824658712
-------------------------------
Converted 8000 cards out of a total of 19264
mins: 26.84776604970296
-------------------------------
Converted 9000 cards out of a total of 19264
mins: 30.140056665738424
--------------------

In [41]:
converted_df['loyalty_back'].value_counts()

3    3
7    1
4    1
5    1
Name: loyalty_back, dtype: int64

In [36]:
converted_df.loc[converted_df['toughness'] == '7-*']

,+0/+1,+0/+1 until,+0/+1 until end,+0/+1 until end of,+0/+1 until end of turn,+0/+2,+0/+2 until,+0/+2 until end,+0/+2 until end of,+0/+2 until end of turn,+0/+3,+1,+1/+0,+1/+0 and,+1/+0 and gains,+1/+0 and has,+1/+0 counters,...,legalities_historic,legalities_modern,legalities_penny,legalities_legacy,legalities_pioneer,legalities_future,legalities_duel,legalities_vintage,legalities_pauper,legalities_brawl,legalities_oldschool,legalities_commander,legalities_standard,rarity_common,rarity_uncommon,rarity_rare,rarity_mythic
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Shapeshifter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [34]:
# convert 'NONE's to np.NaN's so we can scale our data then impute all the NaNs
converted_df = converted_df.replace('NONE', np.NaN)

# convert those columns to numerical data
converted_df['power_back'] = converted_df['power_back'].astype(float)
converted_df['power'] = converted_df['power'].astype(float)
converted_df['toughness'] = converted_df['toughness'].astype(float)
converted_df['toughness_back'] = converted_df['toughness_back'].astype(float)
converted_df['loyalty'] = converted_df['loyalty'].astype(float)
converted_df['loyalty_back'] = converted_df['loyalty_back'].astype(float)

ValueError: could not convert string to float: '7-*'

Now that we have everything coverted to numerical data, we should scale the non-binarized features so everything will be weighted equally. This means we should just scale the columns that have a range of values, e.g. power, toughness, loyalty, and cmc.

In [ ]:
range_col_list = ['power', 'power_back', 'toughness', 'toughness_back', 'loyalty', 'loyalty_back', 'cmc']
converted_df[range_col_list]

In [ ]:
ss = StandardScaler()
range_col_sc = ss.fit_transform(converted_df[range_col_list])

In [ ]:
sc_df = pd.DataFrame(range_col_sc, index = df.index.values, columns=range_col_list)

In [ ]:
sc_df.head()

In [ ]:
sc_df = sc_df.fillna(0)

In [ ]:
sc_df.head()

In [ ]:
converted_df = pd.concat([converted_df.fillna(0).drop(columns=range_col_list), sc_df], axis=1)

In [ ]:
# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

In [ ]:
converted_df.isnull().sum().sum()

In [ ]:
t0 = time.time()
# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')
print((time.time() - t0)/60)

In [ ]:
# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

In [ ]:
# now to test the recommender system
rec_df['Shock'].sort_values()[1:11]

In [ ]:
rec_df['Lightning Bolt'].sort_values()[1:11]

In [ ]:
rec_df['Static Orb'].sort_values()[1:11]

In [ ]:
rec_df['Prized Amalgam'].sort_values()[1:11]

In [ ]:
rec_df['Wrath of God'].sort_values()[1:11]

In [ ]:
rec_df['Jace, the Mind Sculptor'].sort_values()[1:11]

In [ ]:
rec_df['Delver of Secrets // Insectile Aberration'].sort_values()[1:11]

In [ ]:
rec_df['Grizzly Bears'].sort_values()[1:11] # vanilla creature

In [ ]:
rec_df['Oko, Thief of Crowns'].sort_values()[1:11]

In [ ]:
rec_df['Gaze of Granite'].sort_values()[1:11]

In [ ]:
rec_df['Tarmogoyf'].sort_values()[1:11]

In [ ]:
rec_df['Jace, Vryn\'s Prodigy // Jace, Telepath Unbound'].sort_values()[1:11]

In [ ]:
rec_df['Fatal Push'].sort_values()[1:11]

In [ ]:
rec_df['Veil of Summer'].sort_values()[1:11]

In [ ]:
rec_df['Urza, Lord High Artificer'].sort_values()[1:11]

In [ ]:
converted_df = converted_df.round(decimals=3)

In [ ]:
converted_df.head()

In [ ]:
rec_df = rec_df.round(decimals=3)

In [ ]:
rec_df.head()

In [ ]:
converted_df.to_csv('../Data/converted_df.csv')

In [ ]:
rec_df.to_csv('../Data/recommender_df.csv')

In [ ]:
# need to drop supertype, 

drop_list = list(vt_list) + ['rarity_common', 'rarity_uncommon', 'rarity_rare', 'rarity_mythic',
                            'activated_ability', 'triggered_ability', 'cmc', 'power', 'power_back', 'toughness',
                            'toughness_back', 'loyalty', 'loyalty_back', 'super_type_Basic', 'super_type_World',
                            'super_type_Legendary', 'super_type_Snow']
filter_df = converted_df.drop(columns=drop_list)
filter_df['cmc'] = df['cmc']
filter_df.head()

In [ ]:
filter_rec_df = pd.concat([rec_df, filter_df], axis=1)
filter_rec_df.to_pickle('../Data/filter_rec_df.pkl')

### SQL
I can't have an actual online live demo because my tables are too large to host for free. So, I'll being setting up a local version of what the app would be if I could get it to be live. For a local version I'm going to be using SQLite

In [ ]:
# # create a database for our SQL tables
# connection = sqlite3.connect('../Data/MTG_Recommender.db')

# # create our filter table
# filter_df.to_sql('Filter', connection, if_exists='replace')

In [ ]:
# this is where it get's tricky. Normally SQLite won't accept a table with over 2,000 columns so we'll have to
# build it piece by piece
